# Crypto Currency Data From CoinGecko API

## Packages

In [7]:
%pip install ipython-sql prettytable

Note: you may need to restart the kernel to use updated packages.


## Imported Libraries

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import prettytable
import sqlite3

## Log Progress Fuction

In [9]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format='%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./log_progress","a") as f:
        f.write(f"{timestamp}:{message}" + '\n')

## Extract Fuction

This function retrieves cryptocurrency data from the CoinGecko API and saves it to a pandas DataFrame. The returned DataFrame is formatted for further analysis or processing.

Specifically, the function extracts information about the top 100 cryptocurrencies, including their ID, name, current price in USD, and market capitalization in USD.

In [10]:
def extract(url):    # sourcery skip: raise-specific-error
    ''' This function aims to extract the required
    information from the CoinGecko and save it to a data frame. The
    function returns the data frame for further processing. '''
    params = {  
               'vs_currency': 'USD'
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data from API. Status code: {response.status_code}")
    data = response.json()
    df = pd.DataFrame(data)
    columns = ['id', 'name', 'current_price', 'market_cap']
    return df[columns]

## Transform the current price and market cap to EUR and GBP

This function accesses a CSV file containing exchange rate information and adds three new columns to the DataFrame. These columns represent the transformed values of the 'Market Cap' and 'Current Price' columns into their respective currencies.

The function utilizes the `exchange_rate.csv` file to obtain the current exchange rates for USD to EUR and USD to GBP


In [11]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    # Get the exchange rate from the csv file
    exchangerate_df = pd.read_csv(csv_path)
    # Transform the exchange rate in the data frame to a dictionary, in order manipulate it.
    exchange_rate = exchangerate_df.set_index('Currency').to_dict()['Rate']

    # Added new columns
    df['current_price_GBP'] = [np.round(x*exchange_rate['GBP'],2) for x in df['current_price']]
    df['current_price_EUR'] = [np.round(x*exchange_rate['EUR'],2) for x in df['current_price']]
    df['market_cap_GBP'] = [np.round(x*exchange_rate['GBP'],2) for x in df['market_cap']]
    df['market_cap_EUR'] = [np.round(x*exchange_rate['EUR'],2) for x in df['market_cap']]
    return df

## Load Fuctions

The firta function saves the final data frame as a CSV file in
the provided path. Function returns nothing.

The second fuction save saves the final data frame to a database
table with the provided name. Function returns nothing.

Additionally, we include the function `connection_to_database`, which establishes the connection to the database

In [12]:
def load_to_csv(df,new_path):
    df.to_csv(new_path)

def conection_to_database(database_name):
    try:
        db_connection = sqlite3.connect(database_name)
    except sqlite3.OperationalError as e:
        raise e
    else:
        print("connected")
    return db_connection

def load_to_db(df,sql_connection,table_name):
        df.to_sql(table_name,sql_connection, if_exists='replace', index=False)


## ETL Procces

Now that we have all the required functions for this process, we can create a DataFrame based on the data extracted from the CoinGecko API, save it to a CSV file, and load it into a database for subsequent analysis.

### Required variables

In [13]:
api_url = "https://api.coingecko.com/api/v3/coins/markets?x_cg_demo_api_key=CG-MbEY8jPE4gh6VQGrJrCCF5st"
exchange_rate_csv = './exchange_rate.csv'
data_csv_path='./Crypto_Data.csv'
db_name='CryptoData.db'
table_name='Crypto_Data'
log_progress("Variables are define. Intiating ETL process")

Extract data from the api

In [14]:
df = extract(api_url)
df
log_progress("Extracted crypto data from CoinGecko API")

Transforming Data as required. In this case we are going to transform the market cap and current price, which is in USD to EUR and GBP

In [15]:
df = transform(df, exchange_rate_csv)
df

,id,name,current_price,market_cap,current_price_GBP,current_price_EUR,market_cap_GBP,market_cap_EUR
0,bitcoin,Bitcoin,102487.000000,2028043916372,81989.60,97362.65,1.622435e+12,1.926642e+12
1,ethereum,Ethereum,3125.310000,376454750206,2500.25,2969.04,3.011638e+11,3.576320e+11
2,ripple,XRP,3.080000,177459935026,2.46,2.93,1.419679e+11,1.685869e+11
3,tether,Tether,0.999547,139404668728,0.80,0.95,1.115237e+11,1.324344e+11
4,solana,Solana,230.460000,112115003667,184.37,218.94,8.969200e+10,1.065093e+11
...,...,...,...,...,...,...,...,...
95,usual-usd,Usual USD,0.997792,1204780556,0.80,0.95,9.638244e+08,1.144542e+09
96,floki,FLOKI,0.000122,1183897310,0.00,0.00,9.471178e+08,1.124702e+09
97,jasmycoin,JasmyCoin,0.024331,1177639659,0.02,0.02,9.421117e+08,1.118758e+09
98,eos,EOS,0.736423,1116828098,0.59,0.70,8.934625e+08,1.060987e+09


## Loading data

Now we are saving the data to a new CSV file and loading it into the database.

In [16]:
load_to_csv(df,data_csv_path)

This generates a new CSV file in the current working directory containing cryptocurrency data extracted from the CoinGecko API. Next step is going to be load the data in to a Database.

In [17]:
conn = conection_to_database(db_name)
# In order to execute SQL statements and fetch results from SQL queries, we will need to use a database cursor.
curs = conn.cursor()
load_to_db(df,conn,table_name)

connected
